# Step 1 - Subset & Ressampling & Mask w/ IdePix

In [5]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.
 
output_folder = 'l1c_ressampled_masked_validation/'
name_start = 'S2_MSIL1C_'
name_end = '_samp_mask.dim'
input_folder = 'L1C_zip_files_validation/'

def process_zip_files(output_folder, name_start, name_end, input_folder):
    """
    Process the zip files in the input folder by resampling and masking them using the gpt tool.
    
    This function iterates over the zip files in the input folder, extracts the acquisition date from the file name,
    and generates the output file path. It then executes the gpt command to subset, resample and mask the input file.

    """

    for zip_file in os.listdir(input_folder):
        if zip_file.endswith('.zip'):
            product_basename = os.path.basename(zip_file)
            ac_date = product_basename.split('_')[2]
            output_pathname = os.path.join(output_folder, name_start + ac_date + name_end)
            input_pathname = os.path.join(input_folder, zip_file)

            cmd = [ 
                'gpt',
                'data/STEP1_subset_resample_mask_validation.xml',
                '-PInput=' + input_pathname,
                '-POutput=' + output_pathname
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Here we call the function to process the zip files in the input folder.
process_zip_files(output_folder, name_start, name_end, input_folder)

STDOUT: Executing processing graph
Initializing Elevation Model
100% done.

100% done.

100% done.

100% done.

100% done.

100% done.
Initializing Elevation Model
100% done.

100% done.

100% done.

100% done.

100% done.

100% done.

100% done.

100% done.

100% done.

100% done.

100% done.
15%30%....40%..60%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.s2tbx.dataio.s2.ortho.S2OrthoProductReaderPlugIn: Building product reader - EPSG:32631
INFO: org.hsqldb.persist.Logger: 

# Step 2 - Application of C2RCC w/ Meteo Data

In [8]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.

output_folder = 'c2rcc_validation/'
met_data = 'data/validation_data/meteo_data_validation.csv'
param_file = 'data/STEP2_c2rcc_correction_adjusted.xml'
input_folder = 'l1c_ressampled_masked_validation/'

def process_c2rcc_correction(input_folder, output_folder, met_data, param_file):
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """
    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]
            print(idate)
            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the ozone, pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        ozone = parts[3]
                        # Remove the newline character from the ozone value if it exists
                        ozone = ozone.replace('\n', '')
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Pozone=' + ozone,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction(input_folder, output_folder, met_data, param_file)

20231005
STDOUT: Executing operator...
10%20%.51%48%.39%....30%45%36%33%42%......54%...91%... done.
Writing...
....10%....20%...30%...42%...54%...64%...76%...88%...100% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.s3tbx.c2rcc.ancillary.AtmosphericAuxdataBuilder: Atmospheric auxdata product can't be used. At least one is not specified. Using constant values for ozone (300.0) and surface pressure (1014.825).
INFO: org.esa.

## Step 2.1 - Application of C2RCC w/ Meteo Data (excluding Ozone data)

In [ ]:
import os
import subprocess

output_folder = 'c2rcc_without_ozone/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP2_c2rcc_correction.xml'
input_folder = 'l1c_ressampled_masked/'

def process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file):

    
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """

    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]

            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file)

## Step 2.2 - Application of C2RCC w/ Meteo Data + C2-NETS

In [ ]:
import os
import subprocess

output_folder = 'c2rcc_complex_nets/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP22_c2rcc_complex_nets_correction.xml'
old_end = '.dim'
new_end = '_C2RCC.dim'

input_folder = 'l1c_ressampled_masked/'

process_c2rcc_correction(input_folder, output_folder, met_data, param_file)


# Step 3 - Bio-optical models calculation

In [10]:
"""
This script performs 6 bands manipulations of Rrs (Remote Sensing Reflectance) to estimate chlorophyll-a concentration.
It applies different equations to calculate the chlorophyll-a concentration using the Rrs values of different bands.

The following equations are used to calculate the chlorophyll-a concentration using the Rrs values of different bands:

1. 3_bands_model_chl_conc - 161.24*((1/ rrs_B4 - 1/ rrs_B5)/(1/ rrs_B6 - 1/ rrs_B5))+28.04
2. MCI - rrs_B5 - rrs_B4 - (rrs_B6 -rrs_B4)*((709-665)/(754-665))
3. empirical_2_bands_equation1 - 136.3 * (rrs_B6/rrs_B4) - 16.2
4. empirical_2_bands_equation2 - 25.28*((rrs_B5/rrs_B4)*(rrs_B5/rrs_B4)) + 14.85 * (rrs_B5/rrs_B4) - 15.18
5. empirical_3_bands_equation1 - 117.42 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 23.174
6. empirical_3_bands_equation3 - 315.5 * (((1/rrs_B4 - 1/rrs_B5))*rrs_B6)*(((1/rrs_B4 - 1/rrs_B5))*rrs_B6)+ 215.95 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 25.66

The script takes input files from the 'c2rcc' folder, applies the calculations defined in the 'STEP3_chla_algorithms_calculation.xml' graph file,
and saves the output files with modified names in the 'c2rcc_final_bands' folder.

The script uses the 'gpt' command-line tool to execute the graph file and perform the calculations.

Note: The 'gpt' command-line tool must be installed and accessible in the system's PATH for the script to work properly.
"""

import os
import subprocess

output_folder = 'c2rcc_final_validation/'
input_folder = 'c2rcc_validation/'

graph_file = 'data/STEP3_chla_algorithms_calculation.xml'

old_end = '.dim'
new_end = '_mathbands.dim'

for dim_file in os.listdir(input_folder):
    if dim_file.endswith('.dim'):
        dim_file_path = os.path.join(input_folder, dim_file)
        name = os.path.basename(dim_file_path)

        output_name = name.replace(old_end, new_end)
        output_path = os.path.join(output_folder, output_name)

        cmd = [ 
            'gpt',
            graph_file,
            '-PInput=' + dim_file_path,
            '-POutput=' + output_path
        ]

        result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("STDOUT:", result.stdout)
        print("STDERR:", result.stderr)


STDOUT: Executing processing graph
.11%.22%.33%.45%.56%.67%.78%...89% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start

STDOUT: Executing processing graph
.11%.22%.33%.45%.56%.67%.78%...89% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.c

# Step 4 - Extract values of the bands

### Results of C2RCC with Ozone Data - window 3x3

In [11]:
import os
import subprocess

output_folder = 'stats_results_windows_validation'
param_file = 'data/STEP4_1_pixel_values_extraction_3x3.xml'

input_folder = 'c2rcc_final_validation/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

STDOUT: Executing processing graph
...12%..22%..45%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start



### Results of C2RCC with Ozone Data - window 5x5

In [12]:
import os
import subprocess

output_folder = 'stats_results_windows_validation'
param_file = 'data/STEP4_2_pixel_values_extraction_5x5.xml'

input_folder = 'c2rcc_final_validation/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

STDOUT: Executing processing graph
...12%..22%..45%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start



### Consolidation of window results

In [14]:
import pandas as pd

# Read the csv files and skipping the first 6 rows 
df_3x3 = pd.read_csv('stats_results_windows_validation/pixEx_3x3__BandMath_measurements.txt', delimiter='\t', skiprows=6)
df_5x5 = pd.read_csv('stats_results_windows_validation/pixEx_5x5_BandMath_measurements.txt', delimiter='\t', skiprows=6)

# Selecting the columns that we want to analyse

columns = ['Date(yyyy-MM-dd)', '3_bands_model_chl_conc_mean', '3_bands_model_chl_conc_sigma',
           '3_bands_model_chl_conc_num_pixels', 'kd_z90max_mean', 'kd_z90max_sigma', 'kd_z90max_num_pixels',
           'conc_tsm_mean', 'conc_tsm_sigma', 'conc_tsm_num_pixels', 'conc_chl_mean', 'conc_chl_sigma',
           'conc_chl_num_pixels', 'MCI_mean', 'MCI_sigma', 'MCI_num_pixels', 'empirical_2_bands_equation1_mean',
           'empirical_2_bands_equation1_sigma', 'empirical_2_bands_equation1_num_pixels',
           'empirical_2_bands_equation2_mean', 'empirical_2_bands_equation2_sigma',
           'empirical_2_bands_equation2_num_pixels', 'empirical_3_bands_equation1_mean',
           'empirical_3_bands_equation1_sigma', 'empirical_3_bands_equation1_num_pixels',
           'empirical_3_bands_equation3_mean', 'empirical_3_bands_equation3_sigma',
           'empirical_3_bands_equation3_num_pixels']

# Creating a new column with the window size of each row
df_3x3 = df_3x3[columns]
df_3x3['window_size'] = 3
df_5x5 = df_5x5[columns]
df_5x5['window_size'] = 5

# Display DataFrame
display(df_3x3)
display(df_5x5)

# Concatenate the two DataFrames and save the results to a csv file
df_windows = pd.concat([df_3x3, df_5x5], ignore_index=True)
display(df_windows)

df_windows.to_csv('data/validation_data/results_window_calculation.csv', index=False)

,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2023-10-05,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
1,2023-10-07,9.279845,1.160632,9,1.614791,0.274967,9,0.691812,0.020172,9,...,9.767859,1.261423,9,13.759889,0.673879,9,10.383541,0.960083,9,3
2,2023-10-17,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
3,2023-11-19,13.929110,1.798143,9,0.860610,0.114942,9,0.884295,0.349168,9,...,14.974104,1.820038,9,16.492228,1.012330,9,14.413876,1.548207,9,3
4,2023-12-06,4.209487,0.225784,9,38.049126,10.465578,9,0.047429,0.036359,9,...,-0.031206,0.164904,9,10.000746,0.077745,9,5.403882,0.096028,9,3
5,2023-12-16,4.394171,1.212876,9,4.488087,4.681315,9,0.190455,0.182669,9,...,5.330717,2.449425,9,11.093441,0.890837,9,6.798050,1.156926,9,3
6,2024-01-05,8.237055,2.038943,9,1.539977,0.663776,9,0.553170,0.124638,9,...,9.907201,2.798287,9,13.446720,1.332619,9,9.971644,1.865570,9,3


,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2023-10-05,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5
1,2023-10-07,10.451371,1.367539,25,1.361495,0.283574,25,0.711885,0.037740,25,...,11.084144,1.467704,25,14.449390,0.787913,25,11.369831,1.134338,25,5
2,2023-10-17,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5
3,2023-11-19,14.329644,1.922622,25,0.854266,0.127102,25,0.987460,0.348360,25,...,15.286492,1.923441,25,16.694744,1.071532,25,14.729716,1.643379,25,5
4,2023-12-06,4.053040,0.308572,25,34.867924,10.989806,25,0.042164,0.026266,25,...,0.062906,0.261203,25,9.951313,0.107818,25,5.342959,0.132817,25,5
5,2023-12-16,4.203894,1.057289,25,4.569344,3.306057,25,0.137083,0.159098,25,...,4.640472,2.456684,25,10.877993,0.859204,25,6.522068,1.108419,25,5
6,2024-01-05,7.394700,2.557552,25,2.028279,1.189462,25,0.464089,0.247371,25,...,8.625372,3.555693,25,12.868848,1.668371,25,9.198788,2.299667,25,5


,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2023-10-05,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
1,2023-10-07,9.279845,1.160632,9,1.614791,0.274967,9,0.691812,0.020172,9,...,9.767859,1.261423,9,13.759889,0.673879,9,10.383541,0.960083,9,3
2,2023-10-17,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
3,2023-11-19,13.929110,1.798143,9,0.860610,0.114942,9,0.884295,0.349168,9,...,14.974104,1.820038,9,16.492228,1.012330,9,14.413876,1.548207,9,3
4,2023-12-06,4.209487,0.225784,9,38.049126,10.465578,9,0.047429,0.036359,9,...,-0.031206,0.164904,9,10.000746,0.077745,9,5.403882,0.096028,9,3
5,2023-12-16,4.394171,1.212876,9,4.488087,4.681315,9,0.190455,0.182669,9,...,5.330717,2.449425,9,11.093441,0.890837,9,6.798050,1.156926,9,3
6,2024-01-05,8.237055,2.038943,9,1.539977,0.663776,9,0.553170,0.124638,9,...,9.907201,2.798287,9,13.446720,1.332619,9,9.971644,1.865570,9,3
7,2023-10-05,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5
8,2023-10-07,10.451371,1.367539,25,1.361495,0.283574,25,0.711885,0.037740,25,...,11.084144,1.467704,25,14.449390,0.787913,25,11.369831,1.134338,25,5
9,2023-10-17,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5


### Results of C2RCC with Ozone Data - All parameters

In [15]:
import os
import subprocess

output_folder = 'stats_results_windows_validation/'
param_file = 'data/STEP4_3_all_pixels_extraction_StatisticOperator.xml'
input_folder = 'c2rcc_final_validation/'

def process_files(output_folder, param_file, input_folder):
    """
    Process the files in the input folder by running a command line tool that extract 
    the pixel values of bands selected in the parameter file.
    
    The bands selected are: conc_chl, conc_tsm, 3_bands_model_chl_conc, MCI, empirical_2_bands_equation1, 
    empirical_2_bands_equation2, empirical_3_bands_equation1, empirical_3_bands_equation3.

    The output files will be .txt and saved in the output folder.
    """

    old_end = '.dim'
    new_end = '_stats.txt'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                param_file,
                '-PInput=' + dim_file_path,
                '-POutput='+ output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

process_files(output_folder, param_file, input_folder)


STDOUT: Executing processing graph
30%60%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.snap.statistics.ProductLoop:     current product: c:\Users\arthu\Documents\GitHub\remote_sensing_lac_creteil\c2rcc_final_validation\S2_MSIL1C_20231005T105911_samp_mask_C2RCC_mathbands.dim
INFO: org.esa.snap.statistics.StatisticsOp: Successfully computed statistics.

STDOUT: Executing processing graph
30%60%90% done.

STDERR: INFO: o

In [16]:
import pandas as pd
import os

def process_data(input_folder):
    """
    Process the data files in the given input folder.
    
    Args:
        input_folder (str): Path to the folder containing the data files.
        
    Returns:
        pd.DataFrame: Consolidated dataframe containing the processed data.
    """
    # Create an empty list to store the dataframes
    dfs = []

    # Iterate over the files in the input folder
    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('_stats.txt'):
            # Extract the date from the file name
            name = os.path.basename(dim_file)
            idate = name.split('_')[2].split('T')[0]

            # Read the data from the file into a dataframe
            df_virtual = pd.read_csv(os.path.join(input_folder, dim_file), sep='\t')
            df_virtual['date'] = pd.to_datetime(idate, format='%Y%m%d')

            # Append the dataframe to the list
            dfs.append(df_virtual)

    # Concatenate all the dataframes into a single dataframe
    consolidated_df = pd.concat(dfs, ignore_index=True)

    return consolidated_df

# Running the function
input_folder = 'stats_results_windows_validation/'
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv('data/validation_data/results_final_bands.csv', index=False)


,name,Band,average,max_error,maximum,median,minimum,p80_threshold,p85_threshold,p90_threshold,sigma,total,date
0,world,3_bands_model_chl_conc,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-10-05
1,world,MCI,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,8800,2023-10-05
2,world,conc_chl,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-10-05
3,world,conc_tsm,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-10-05
4,world,empirical_2_bands_equation1,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-10-05
5,world,empirical_2_bands_equation2,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-10-05
6,world,empirical_3_bands_equation1,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-10-05
7,world,empirical_3_bands_equation3,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-10-05
8,world,3_bands_model_chl_conc,9.6601,0.0143,17.4273,10.5406,3.0801,12.5205,12.8505,13.2953,3.3031,790,2023-10-07
9,world,MCI,0.0000,0.0000,0.0002,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,8800,2023-10-07
